In [1]:
# to do:
# ask about Volcano plots

In [2]:
import numpy as np
import pandas as pd
import sys
import csv
import os
import matplotlib.pyplot as plt
from scipy.stats import *
from tabulate import tabulate
import warnings
import time
import math
import seaborn as sns
import random
import pprint
import statsmodels.formula.api as smf
import statsmodels.api as sm
from openpyxl import load_workbook
from scipy.interpolate import griddata

In [3]:
plt.rcParams['figure.dpi'] = 1000
plt.rcParams['font.family'] = 'Times New Roman'

In [4]:
start_time = time.time()

In [5]:
path_in = '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/input_output/9_10_25_test/'
path_out = '/Users/kateschnee/Desktop/tmp/'

output_dir = os.path.join(path_out, 'Supplemental figures')
os.makedirs(output_dir, exist_ok=True)

In [6]:
def progress_meter(i, j, e):
    print(f'{e}: {math.ceil(100 * i /j)}%', end = '\r')

## Load data sets

In [7]:
# read these as csv, specially mark experiment and barcode
dfe = pd.read_csv(path_in + 'dfe.csv', index_col = ['experiment', 'barcode'])
dfs = pd.read_csv(path_in + 'dfs.csv', index_col = ['experiment', 'barcode'])

In [8]:
experiments = list(dfe.index.get_level_values('experiment').unique())
t = 1.1
comparisons = pd.read_excel(path_in + 'input_list_of_comparisons.xlsx')

In [9]:
# issues here where function doesn't modify inplace if you don't do inplace = True
dfe.fillna(0, inplace = True)
dfs.fillna(0, inplace = True)

In [10]:
def stroma_vs_epi_map(epi_df, stroma_df, show_output = False, dot_size = 1, suffix = ''):
    plt.figure(figsize = (5,5))
    plt.scatter(x = epi_df['pxl_col_in_fullres_non_gene'].astype(float), y = epi_df['pxl_row_in_fullres_non_gene'].astype(float),
               color = 'blue', label = 'epithelium', s = 1)
    plt.scatter(x = stroma_df['pxl_col_in_fullres_non_gene'].astype(float), y = stroma_df['pxl_row_in_fullres_non_gene'].astype(float),
               color = 'red', label = 'stroma', s = dot_size)
    plt.title(experiment + suffix)
    plt.legend(bbox_to_anchor=(0.5, -0.1), loc='upper center', ncol=2)
    plt.savefig(path_out + 'Supplemental figures/' + experiment + suffix + '_epi_vs_stroma.tif',
               bbox_inches='tight')

    if show_output:
        plt.show()
    else:
        plt.close()

In [11]:
def get_arrow_coords(df_base, df_ref):
    coord_pairs = list(zip(list(df_base.index.values), list(df_base['partner_barcode_non_gene'].values)))

    base_coords = []
    reference_coords = []

    for pair in coord_pairs:
        # try:
        base_barcode = pair[0]
        reference_barcode = pair[1]
        if base_barcode in df_base.index.values and reference_barcode in df_ref.index.values:
            base_coords.append([df_base.loc[base_barcode, 'pxl_col_in_fullres_non_gene'].astype(float),
                                df_base.loc[base_barcode, 'pxl_row_in_fullres_non_gene'].astype(float)])
            reference_coords.append([df_ref.loc[reference_barcode, 'pxl_col_in_fullres_non_gene'].astype(float),
                                df_ref.loc[reference_barcode, 'pxl_row_in_fullres_non_gene'].astype(float)])
        # except:
        #     print(f'Experiment: {experiment} Base barcode: {base_barcode}   specified partner: {reference_barcode}')

    x_base, y_base = zip(*base_coords)
    x_ref, y_ref = zip(*reference_coords)
    return x_base, y_base, x_ref, y_ref

In [12]:
def arrow_map(df_base, df_ref,show_output = False, dot_size = 4, suffix = ''):

    x_base, y_base, x_ref, y_ref = get_arrow_coords(df_base, df_ref)
    plt.figure(figsize = (12,12))
    plt.scatter(x = dfe_tmp['pxl_col_in_fullres_non_gene'],
                y = dfe_tmp['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'blue',
                label = 'epithelium'
    )
    plt.scatter(x = dfs_tmp['pxl_col_in_fullres_non_gene'],
                y = dfs_tmp['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'red',
                label = 'stroma'
    )
    for i in range(len(x_base)):
        plt.annotate('', xy = (x_ref[i], y_ref[i]), xytext = (x_base[i], y_base[i]), arrowprops = dict(arrowstyle='->', color = 'black'))


    plt.legend(bbox_to_anchor=(0.5, -0.05), loc='upper center', ncol=2)
    plt.title(experiment)
    plt.savefig(path_out + 'Supplemental figures/' + experiment + suffix + '_arrow_map.tif', bbox_inches='tight')
    
    if show_output:
        plt.show()
    else:
        plt.close()

In [13]:
def transparent_plot(dfe_included, dfe_excluded, dfs_included, dfs_excluded, show_output = False, dot_size = 4, suffix = ''):

    #stroma_df_included = stroma_df[stroma_df['barcode'].isin(bc_df['stroma_bc'])]
    #stroma_df_excluded = stroma_df[~stroma_df['barcode'].isin(bc_df['stroma_bc'])]
    plt.figure(figsize = (12,12))
    plt.scatter(x = dfe_included['pxl_col_in_fullres_non_gene'],
                y = dfe_included['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'blue',
                label = 'epithelium_included',
                alpha = 1
    )
    plt.scatter(x = dfe_excluded['pxl_col_in_fullres_non_gene'],
                y = dfe_excluded['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'blue',
                label = 'epithelium_excluded',
                alpha = .1
    )
    plt.scatter(x = dfs_included['pxl_col_in_fullres_non_gene'],
                y = dfs_included['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'red',
                label = 'stroma_included',
                alpha = 1
    )
    plt.scatter(x = dfs_excluded['pxl_col_in_fullres_non_gene'],
                y = dfs_excluded['pxl_row_in_fullres_non_gene'],
                s = dot_size,
                color = 'red',
                label = 'stroma_excluded',
                alpha = .1
    )

    if 'arrow' in suffix:
        if 'epithelium' in suffix:
            x_base, y_base, x_ref, y_ref = get_arrow_coords(dfe_included, dfs_included)
        else:
            x_base, y_base, x_ref, y_ref = get_arrow_coords(dfs_included, dfe_included)
        for i in range(len(x_base)):
            plt.annotate('', xy = (x_ref[i], y_ref[i]), xytext = (x_base[i], y_base[i]), arrowprops = dict(arrowstyle='->', color = 'black'))

    plt.legend(bbox_to_anchor=(0.5, -0.05), loc='upper center', ncol=2)
    plt.title(experiment)
    plt.savefig(path_out + 'Supplemental figures/' + experiment + suffix + '_transparent_map.tif',
               bbox_inches='tight')
    if show_output:
        plt.show()
    else:
        plt.close()

In [14]:
# images
d = []
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for experiment in experiments:
        # .loc() returns tmp df with all the rows of a specific experiment
        dfe_tmp = dfe.loc[experiment]
        dfs_tmp = dfs.loc[experiment]

        # basically just makes sure that partner stroma/epithilial point is in this experiment (it should be)
        dfe_tmp = dfe_tmp[dfe_tmp['partner_barcode_non_gene'].isin(dfs_tmp.index)]
        dfs_tmp = dfs_tmp[dfs_tmp['partner_barcode_non_gene'].isin(dfe_tmp.index)]

        stroma_vs_epi_map(dfe_tmp, dfs_tmp)

        # plot arrow map for each epithelial spot
        arrow_map(dfe_tmp, dfs_tmp, suffix = '_epithelium_as_base')
        min_dist = dfe_tmp['distance_non_gene'].min()
        arrow_map(dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t], dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t], suffix = '_epithelium_as_base_filtered')
        transparent_plot(dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t],
                         dfe_tmp[dfe_tmp['distance_non_gene'] > min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] > min_dist * t],
                         suffix = '_epithelium_as_base'
                        )
        transparent_plot(dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t],
                         dfe_tmp[dfe_tmp['distance_non_gene'] > min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] > min_dist * t],
                         suffix = '_epithelium_as_base_arrow'
                        )


        # now reverse
        arrow_map(dfs_tmp, dfe_tmp, suffix = '_stroma_as_base')
        min_dist = dfs_tmp['distance_non_gene'].min()
        arrow_map(dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t], dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t], suffix = '_stroma_as_base_filtered')
        transparent_plot(dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t],
                         dfe_tmp[dfe_tmp['distance_non_gene'] > min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] > min_dist * t],
                         suffix = '_stroma_as_base'
                        )
        transparent_plot(dfe_tmp[dfe_tmp['distance_non_gene'] <= min_dist * t],
                         dfe_tmp[dfe_tmp['distance_non_gene'] > min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] <= min_dist * t],
                         dfs_tmp[dfs_tmp['distance_non_gene'] > min_dist * t],
                         suffix = '_stroma_as_base_arrow'
                        )


In [15]:
def spearman_different(df_base, df_reference, gene_input):

    # remove any errors for keys

    # ensures that df_base only contains points that are validly paired with a stromal/other point
    base_tuples = list(zip(df_base.index.get_level_values('experiment'), df_base['partner_barcode_non_gene']))
    df_base = df_base[pd.Series(base_tuples).isin(df_reference.index).values]

    # contains the data from df_reference, but is perfectly ordered and indexed to match the filtered df_base
    df_reference_aligned = df_reference.loc[pd.MultiIndex.from_arrays([
        df_base.index.get_level_values("experiment"),
        df_base["partner_barcode_non_gene"]
    ])].set_index(df_base.index)

    # don't do calculations with non gene columns
    cols_to_avoid = []
    for c in df_base.columns:
        if c.endswith('_non_gene'):
            cols_to_avoid.append(c)
        elif 'barcode' in c:
            cols_to_avoid.append(c)
    cols_to_avoid.append('experiment')

    results = []
    for gene in df_base.columns:
        if gene not in cols_to_avoid:
            try:
                rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])
                results.append([gene, rho, pval, pval * df_base.shape[0]])
            except Exception as e:
                results.append([gene, 'NA', 'NA', 'NA'])
    results_df = pd.DataFrame(results)
    results_df.columns = ['gene', 'rho', 'p_val', 'adj_p_val']
    results_df.set_index('gene', inplace = True)
    results_df.sort_values(by=['rho', 'adj_p_val'], ascending=[False, True], inplace = True)
    #results_df.to_excel(path_out + 'new_alignment_correlations2.xlsx')
    return results_df

In [16]:
def spearman_same(df, gene_input):
    """
    Compute Spearman correlations of gene_input with all other genes in the same region.
    Parameters:
    df : pd.DataFrame, DataFrame with genes as columns. MultiIndex expected with levels ['experiment', 'barcode'].
    gene_input : str, The reference gene to compute correlations against.
    Returns:
    results_df : pd.DataFrame, DataFrame indexed by 'gene', with columns ['rho', 'p_val', 'adj_p_val'].
    """
    cols_to_avoid = []
    for c in df.columns:
        if c.endswith('_non_gene'):
            cols_to_avoid.append(c)
        elif 'barcode' in c:
            cols_to_avoid.append(c)
    cols_to_avoid.append('experiment')
    results = []
    for gene in df.columns:
        if gene not in cols_to_avoid:
            try:
                rho, pval = spearmanr(df[gene], df[gene_input])
                results.append([gene, rho, pval, pval * df.shape[0]])
            except Exception as e:
                results.append([gene, 'NA', 'NA', 'NA'])
    results_df = pd.DataFrame(results)
    results_df.columns = ['gene', 'rho', 'p_val', 'adj_p_val']
    results_df.set_index('gene', inplace = True)
    results_df.sort_values(by=['rho', 'adj_p_val'], ascending=[False, True], inplace = True)
    return results_df

In [17]:
#results_df2 = spearman_different(dfe, dfs, 'alignment')

In [18]:
comparisons

,base_region,reference_region,reference_gene,summary
0,epithelium,stroma,alignment,This will test alignment in the stroma versus ...
1,stroma,epithelium,DDR1,This will test DDR1 in the epithelium versus e...
2,stroma,epithelium,TGFBI,This will test TGFBI in the epithelium versus ...
3,epithelium,epithelium,DDR1,This will test DDR1 in the epithelium versus e...
4,stroma,stroma,alignment,This will test alignment in the stroma versus ...


In [19]:
def split_df_by_experiment(df_master):
    """
    Parameter:
    Dataframe to be segmented (either dfe or dfs)
    Return:
    Dictionary with key experiment and value df segmented by experiment
    EX) {TR11_206: {dfe with rows only for this sample}}
    """
    df_segmented = {}
    for experiment in experiments:
        df_segmented[experiment] = df_master.loc[df_master.index.get_level_values('experiment') == experiment]
    return df_segmented

In [20]:
'''
Segment the dataframes for the stroma and epithilium by sample
    (format is dict of dfs)
For each comparison, call the correct spearman analysis for each sample's df
and store the df and metadata within a dict of all the segmented data
    
    Note: ConstantInputWarning is because our mock data has so many zeroes, so in some there is no variation
    Note: pprint() function doesn't preserve order (which is why dataframe prints before summary)
{
    "experiment_tag": {
        test_number: {
            "base_region": "epithelium",
            "reference_region": "stroma",
            "reference_gene": "ABC",
            "summary": "Comparing ABC in the stroma to every gene in the epithelium",
            "data": <some DataFrame>
        }, etc 
    }
}
'''

all_exp_dicts = {exp: {} for exp in experiments}
    
dfe_segmented = split_df_by_experiment(dfe)
dfs_segmented = split_df_by_experiment(dfs)

for test_num, row in comparisons.iterrows():
    if row['base_region'] == 'stroma':
        if row['reference_region'] == 'stroma':
            # here exp is a key to access each segmented dataframe
            for exp in experiments:
                all_exp_dicts[exp][test_num] = {
                    "base_region": row['base_region'],
                    "reference_region": row['reference_region'],
                    "reference_gene": row['reference_gene'],
                    "summary": f"Comparing {row['reference_gene']} in the {row['reference_region']} to every gene in the {row['base_region']}",
                    "data": spearman_same(dfs_segmented[exp], row["reference_gene"])
                }
        elif row['reference_region'] == 'epithelium':
            for exp in experiments:
                all_exp_dicts[exp][test_num] = {
                    "base_region": row['base_region'],
                    "reference_region": row['reference_region'],
                    "reference_gene": row['reference_gene'],
                    "summary": f"Comparing {row['reference_gene']} in the {row['reference_region']} to every gene in the {row['base_region']}",
                    "data": spearman_different(dfs_segmented[exp], dfe_segmented[exp], row["reference_gene"])
                }

    else:
        if row['reference_region'] == 'epithelium':
            # here exp is a key to access each segmented dataframe
            for exp in experiments:
                all_exp_dicts[exp][test_num] = {
                    "base_region": row['base_region'],
                    "reference_region": row['reference_region'],
                    "reference_gene": row['reference_gene'],
                    "summary": f"Comparing {row['reference_gene']} in the {row['reference_region']} to every gene in the {row['base_region']}",
                    "data": spearman_same(dfe_segmented[exp], row["reference_gene"])
                }
        else:
            for exp in experiments:
                all_exp_dicts[exp][test_num] = {
                    "base_region": row['base_region'],
                    "reference_region": row['reference_region'],
                    "reference_gene": row['reference_gene'],
                    "summary": f"Comparing {row['reference_gene']} in the {row['reference_region']} to every gene in the {row['base_region']}",
                    "data": spearman_different(dfe_segmented[exp], dfs_segmented[exp], row["reference_gene"])
                }


/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])
/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])
/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])
/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])


In [21]:
'''
Output: Separate Excel files with sheets for each comparison.
Writes metadata (base region, summary, etc) in columns 6-7
'''
for exp, tests in all_exp_dicts.items():
    # Create an ExcelWriter for this experiment
    file_path = f"{path_out}{exp}_spearman_data_STE.xlsx"
    with pd.ExcelWriter(file_path) as writer:
        
        for test_num, test_info in tests.items():     
            sheet_name = f"{test_info['reference_gene'][:20]}_{test_num}"
            # Excel sheet names are limited to 31 chars, so truncate if needed
            sheet_name = sheet_name[:31]            
            df = test_info['data']
            # Write dataframe to its own sheet
            df.to_excel(writer, sheet_name=sheet_name)
        
    # Reopen the file for metadata writing
    wb = load_workbook(file_path)  
    for test_num, test_info in tests.items():
        sheet_name = f"{test_info['reference_gene'][:20]}_{test_num}"[:31]
        ws = wb[sheet_name]
        # Write metadata starting in column 6
        ws.cell(row=1, column=6, value="Base Region")
        ws.cell(row=1, column=7, value=test_info['base_region'])
        ws.cell(row=2, column=6, value="Reference Region")
        ws.cell(row=2, column=7, value=test_info['reference_region'])
        ws.cell(row=3, column=6, value="Reference Gene")
        ws.cell(row=3, column=7, value=test_info['reference_gene'])
        ws.cell(row=4, column=6, value="Summary")
        ws.cell(row=4, column=7, value=test_info['summary'])
    # Save metadata changes
    wb.save(file_path)
    print(f"✅ Saved workbook for {exp}: {file_path}")

✅ Saved workbook for TR11_206: /Users/kateschnee/Desktop/tmp/TR11_206_spearman_data_STE.xlsx
✅ Saved workbook for TR11_16184_2: /Users/kateschnee/Desktop/tmp/TR11_16184_2_spearman_data_STE.xlsx
✅ Saved workbook for TR11_21723: /Users/kateschnee/Desktop/tmp/TR11_21723_spearman_data_STE.xlsx
✅ Saved workbook for TR16_23542: /Users/kateschnee/Desktop/tmp/TR16_23542_spearman_data_STE.xlsx
✅ Saved workbook for TR11_18105: /Users/kateschnee/Desktop/tmp/TR11_18105_spearman_data_STE.xlsx


In [22]:
'''
Output: one Excel file with sheets for each comparison.
Runs spearman analysis on all samples combined. 
'''
results = {}
i = 0
test = 0
for i, r in comparisons.iterrows():
    
    test += 1
    results.update({test: {}})
    print(test)
    if r['base_region'] == 'epithelium':
        if r['reference_region'] == 'epithelium':
            df = spearman_same(dfe, r['reference_gene'])
        else:
            
            df = spearman_different(dfe, dfs, r['reference_gene'])
    else:
        if r['reference_region'] == 'epithelium':
            df = spearman_different(dfs, dfe, r['reference_gene'])
        else:
            df = spearman_same(dfs, r['reference_gene'])
    results[test]['base_region'] = r['base_region']
    results[test]['reference_region'] = r['reference_region']
    results[test]['reference_gene'] = r['reference_gene']
    results[test]['summary'] = f"Comparing {r['reference_gene']} in the {r['reference_region']} to every gene in the {r['base_region']}"
    results[test]['data'] = df

with pd.ExcelWriter(path_out + 'spearman_data_STE.xlsx') as writer:
    for test, info in results.items():

        info['data'].to_excel(writer, sheet_name = info['reference_gene'] + '_' + str(test), index = True)

# Load the workbook and select the active sheet
excel_file = path_out + 'spearman_data_STE.xlsx'
wb = load_workbook(excel_file)
i = 0
for ws in wb.worksheets:
    print(f"Processing sheet: '{ws.title}'")

    for j in range(4):
        ws.cell(row=j+1, column=6).value = comparisons.columns[j]
        ws.cell(row=j+1, column=7).value = comparisons.iloc[i, j]
    i += 1
# Save the workbook with the changes
wb.save(excel_file)

1


/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])


2


/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])


3


/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2930291389.py:28: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df_base[gene], df_reference_aligned[gene_input])


4


/var/folders/63/c7hqsqnn4jg009ky06cw0kmh0000gn/T/ipykernel_14766/2144433804.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(df[gene], df[gene_input])


5
Processing sheet: 'alignment_1'
Processing sheet: 'DDR1_2'
Processing sheet: 'TGFBI_3'
Processing sheet: 'DDR1_4'
Processing sheet: 'alignment_5'


In [23]:
end_time = time.time()
print(f'Total runtime: {math.floor((end_time - start_time)/60)} minutes and {math.floor((end_time - start_time)%60)} seconds.')

Total runtime: 6 minutes and 9 seconds.


In [24]:
def three_d_plot(df_base, df_reference):
    # remove any errors for keys

    base_tuples = list(zip(df_base.index.get_level_values('experiment'), df_base['partner_barcode_non_gene']))
    df_base = df_base[pd.Series(base_tuples).isin(df_reference.index).values]

    df_reference_aligned = df_reference.loc[pd.MultiIndex.from_arrays([
        df_base.index.get_level_values("experiment"),
        df_base["partner_barcode_non_gene"]
    ])].set_index(df_base.index)

    fig = plt.figure(figsize = (5,5))
    ax = fig.add_subplot(projection='3d')

    ax.scatter(df_base['TGFBI'], df_base['DDR1'], df_reference_aligned['alignment'], marker='o', s = 1)

    ax.set_xlabel('TGFBI')
    ax.set_ylabel('DDR1')
    ax.set_zlabel('alignment')
    plt.close()




    # Define a regular grid over the full extent of the x and y data
    grid_x, grid_y = np.mgrid[min(df_base['TGFBI']):max(df_base['TGFBI']):100j,
                              min(df_base['DDR1']):max(df_base['DDR1']):100j]

    # 3. Interpolate the Z-values onto the new grid
    # The 'linear' method is often a good starting point for interpolation
    grid_z = griddata((df_base['TGFBI'], df_base['DDR1']), df_reference_aligned['alignment'], (grid_x, grid_y), method='linear')

    # 4. Create the surface plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the surface
    ax.plot_surface(grid_x, grid_y, grid_z, cmap='viridis')
    plt.close()
    df_reference_aligned_1 = df_reference_aligned['alignment']
    new_df = df_base[['TGFBI', 'DDR1']]
    new_df_1 = new_df.join(df_reference_aligned_1, how = 'left')
    new_df_1.to_csv(path_out + 'TGFBI_DDR1_alignment_data.csv')
    new_df_1['experiment'] = new_df_1.index.get_level_values('experiment')
    fig = plt.figure(figsize=(10, 8))
    sns.stripplot(data=new_df_1, x='DDR1', y='TGFBI', hue='alignment', jitter = 1, size = 1)

    results_lm = smf.mixedlm('alignment ~ DDR1 * TGFBI', data = new_df_1, groups = new_df_1['experiment']).fit()
    print(results_lm.summary())

In [25]:
three_d_plot(dfe, dfs)

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: alignment
No. Observations: 5240    Method:             REML     
No. Groups:       5       Scale:              0.0404   
Min. group size:  431     Log-Likelihood:     942.1893 
Max. group size:  1581    Converged:          Yes      
Mean group size:  1048.0                               
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.163    0.055  2.956 0.003  0.055  0.270
DDR1          0.019    0.005  3.486 0.000  0.008  0.030
TGFBI         0.009    0.002  4.049 0.000  0.005  0.013
DDR1:TGFBI   -0.004    0.002 -2.559 0.010 -0.008 -0.001
Group Var     0.015    0.053                           



In [26]:
end_time = time.time()
print(f'Total runtime: {math.floor((end_time - start_time)/60)} minutes and {math.floor((end_time - start_time)%60)} seconds.')

Total runtime: 7 minutes and 47 seconds.


In [27]:
def effects_models_different(df_base, df_reference, genes_to_test):
    base_tuples = list(zip(df_base.index.get_level_values('experiment'), df_base['partner_barcode_non_gene']))
    df_base = df_base[pd.Series(base_tuples).isin(df_reference.index).values]

    df_reference_aligned = df_reference.loc[pd.MultiIndex.from_arrays([
        df_base.index.get_level_values("experiment"),
        df_base["partner_barcode_non_gene"]
    ])].set_index(df_base.index)

    df_reference_aligned = df_reference_aligned['alignment']

    new_df_1 = df_base.join(df_reference_aligned, how = 'left', rsuffix = '_stroma')

    new_df_1['experiment'] = new_df_1.index.get_level_values('experiment')

    output_list = []

    failed_tests = 0
    genes_tested = []
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for geneA in genes_to_test:
            i = 0
            genes_tested.append(geneA)
            for geneB in genes_to_test:
                if geneA != geneB and geneB not in genes_tested:

                    formula = 'DDR1' + ' ~ ' + str(geneA) + ' * ' + str(geneB)
                    tmp_list = [formula]
                    #print(formula)
                    try:
                        results_lm = smf.mixedlm(formula, data = new_df_1, groups = new_df_1['experiment']).fit()
                        coefficients = results_lm.fe_params
                        p_values = results_lm.pvalues

                        # Define the variables of interest
                        variables_of_interest = [geneA, geneB, str(geneA) + ':' + str(geneB)]



                        # Loop through each variable and append the name, coefficient, and p-value
                        for var_name in variables_of_interest:
                            if var_name in coefficients.index:
                                tmp_list.append(var_name)
                                tmp_list.append(coefficients.loc[var_name])
                                tmp_list.append(p_values.loc[var_name])
                                tmp_list.append(p_values.loc[var_name] * new_df_1.shape[0] ** 2)
                        output_list.append(tmp_list)
                    except Exception as e:
                        # print(f'Could not analyze {formula}')
                        for j in range(12):
                            tmp_list.append(np.nan)
                        failed_tests += 1
                i += 1
                progress_meter(i, len(genes_to_test), 'DDR1' + ' ~ ' + str(geneA) + ' * ...')
            print()
    results_df = pd.DataFrame(output_list)
    results_df.columns = ['formula', 'variable1', 'variable1_coeff', 'variable1_p_val', 'variable1_adj_p_val', 'variable2', 'variable2_coeff', 'variable2_p_val', 'variable2_adj_p_val', 'interaction', 'interaction_coeff', 'interaction_p_val', 'interaction_adj_p_val']
    results_df.set_index('formula', inplace = True)
    results_df.dropna(axis = 0, inplace = True)
    results_df.sort_values(by=['variable1_p_val', 'variable2_p_val', 'interaction_p_val', 'variable1_coeff', 'variable2_coeff', 'interaction_coeff'], ascending=[True, True, True, False, False, False], inplace = True)
    return results_df


In [28]:
def effects_model_same(df_in, genes_to_test):
    output_list = []
    df_in['experiment'] = df_in.index.get_level_values('experiment')
    failed_tests = 0
    genes_tested = []
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for geneA in genes_to_test:
            i = 0
            genes_tested.append(geneA)
            for geneB in genes_to_test:
                if geneA != geneB and geneB not in genes_tested:

                    formula = 'alignment' + ' ~ ' + str(geneA) + ' * ' + str(geneB)
                    tmp_list = [formula]
                    #print(formula)
                    try:
                        results_lm = smf.mixedlm(formula, data = df_in, groups = df_in['experiment']).fit()
                        coefficients = results_lm.fe_params
                        p_values = results_lm.pvalues

                        # Define the variables of interest
                        variables_of_interest = [geneA, geneB, str(geneA) + ':' + str(geneB)]



                        # Loop through each variable and append the name, coefficient, and p-value
                        for var_name in variables_of_interest:
                            if var_name in coefficients.index:
                                tmp_list.append(var_name)
                                tmp_list.append(coefficients.loc[var_name])
                                tmp_list.append(p_values.loc[var_name])
                                tmp_list.append(p_values.loc[var_name] * df_in.shape[0] ** 2)
                        output_list.append(tmp_list)
                    except Exception as e:
                        #print(f'Could not analyze {formula}')
                        for j in range(12):
                            tmp_list.append(np.nan)
                        failed_tests += 1
                i += 1
                progress_meter(i, len(genes_to_test), 'alignment' + ' ~ ' + str(geneA) + ' * ...')
            print()
    results_df = pd.DataFrame(output_list)
    results_df.columns = ['formula', 'variable1', 'variable1_coeff', 'variable1_p_val', 'variable1_adj_p_val', 'variable2', 'variable2_coeff', 'variable2_p_val', 'variable2_adj_p_val', 'interaction', 'interaction_coeff', 'interaction_p_val', 'interaction_adj_p_val']
    results_df.set_index('formula', inplace = True)
    results_df.dropna(axis = 0, inplace = True)
    results_df.sort_values(by=['variable1_p_val', 'variable2_p_val', 'interaction_p_val', 'variable1_coeff', 'variable2_coeff', 'interaction_coeff'], ascending=[True, True, True, False, False, False], inplace = True)
    return results_df

In [29]:
# x = effects_models(dfe, dfs)
# x.to_csv(path_out + 'mixed_lm_data.csv')

In [30]:
alignment_df = results[5]['data'][results[5]['data']['adj_p_val'] < .05]
alignment_df.reset_index(inplace = True)
ddr1_df = results[1]['data'][results[1]['data']['adj_p_val'] < .05]
ddr1_df.reset_index(inplace = True)
mixed_lm_df = pd.concat([alignment_df, ddr1_df], axis = 0)
genes_to_test = mixed_lm_df['gene'].unique()[:20]

In [31]:
mixedlm_DDR1 = effects_models_different(dfe, dfs, genes_to_test)
mixedlm_DDR1.to_csv(path_out + 'DDR1_effects_models.csv')

DDR1 ~ alignment * ...: 100%
DDR1 ~ MT-CYB * ...: 100%
DDR1 ~ MT-ND3 * ...: 100%
DDR1 ~ MT-ND2 * ...: 100%
DDR1 ~ MT-ND5 * ...: 100%
DDR1 ~ MT-ND6 * ...: 100%
DDR1 ~ MT-ND1 * ...: 100%
DDR1 ~ MT-ND4L * ...: 100%
DDR1 ~ MT-CO3 * ...: 100%
DDR1 ~ MT-ATP6 * ...: 100%
DDR1 ~ MT-CO2 * ...: 100%
DDR1 ~ MT-ND4 * ...: 100%
DDR1 ~ USP17L3 * ...: 100%
DDR1 ~ USP17L15 * ...: 100%
DDR1 ~ USP17L1 * ...: 100%
DDR1 ~ HEYL * ...: 100%
DDR1 ~ TAF11L4 * ...: 100%
DDR1 ~ CRABP2 * ...: 100%
DDR1 ~ CDKN2B * ...: 100%
DDR1 ~ COL16A1 * ...: 100%


In [32]:
mixedlm_alignment = effects_model_same(dfs, genes_to_test)
mixedlm_alignment.to_csv(path_out + 'alignment_effects_models.csv')

alignment ~ alignment * ...: 100%
alignment ~ MT-CYB * ...: 100%
alignment ~ MT-ND3 * ...: 100%
alignment ~ MT-ND2 * ...: 100%
alignment ~ MT-ND5 * ...: 100%
alignment ~ MT-ND6 * ...: 100%
alignment ~ MT-ND1 * ...: 100%
alignment ~ MT-ND4L * ...: 100%
alignment ~ MT-CO3 * ...: 100%
alignment ~ MT-ATP6 * ...: 100%
alignment ~ MT-CO2 * ...: 100%
alignment ~ MT-ND4 * ...: 100%
alignment ~ USP17L3 * ...: 100%
alignment ~ USP17L15 * ...: 100%
alignment ~ USP17L1 * ...: 100%
alignment ~ HEYL * ...: 100%
alignment ~ TAF11L4 * ...: 100%
alignment ~ CRABP2 * ...: 100%
alignment ~ CDKN2B * ...: 100%
alignment ~ COL16A1 * ...: 100%
